# Introduction

The goal of this project is to develop a library for performing automatic differentiation (AD), which we will call ```LazyDiff```.
AD is a set of techniques to numerically evaluate the derivative of a function motivated by deficiencies in classical methods. Numerical approximation is often not accurate enough and can be computationally expensive and symbolic differentiation tends to lead to inefficient code and faces the difficulty of converting a computer program into a single expression. Both classical methods have problems with calculating higher derivatives, where the complexity and errors increase, and are slow when it comes to computing the partial derivatives of a function with respect to many inputs. Automatic differentiation endeavors to solve all of these problems, and has been used in neural networks for back propagation and weight adjustment based on the loss function, and in scientific computing where it is often difficult to analytically compute the derivatives.





# How to Use LazyDiff

## How to Install

If you want to install the package only for the current virtual environment, run the following commands to create and activate a new virtual env:

- Move to the desired working directory
- `virtualenv env`
- `source env/bin/activate`

You can run the following to install the dependencies:
- `pip install -r requirements.txt`

Then execute following command to install LazyDiff and its dependencies:
 - `pip install git+https://github.com/CS207-Project-Group-7/cs207-FinalProject.git`
 
 
 ## Basic Demo
 
 ### Variable Objects and Evaluating Functions
 
 The values of variables are stored inside `vars` objects as `Scalar` or `Vector`, which are used as function inputs. Custom functions from `ops` module need to be used to evaluate the functions, which support all the standard math functions from math and numpy libraries. 
 
 Function values can be evaluated by simply wrapping the variables with the corresponding functions and accessing the values using `.val`
 
Below we show a simple example for evaluating $f(x) = x^2$ and $f(y) = sin(y)$ for variables $x=3$ and $y=0$. We also show how we can compound the functions by evaluating $f(z) = 5*z$ where $z = f(x)$.
 


In [1]:
from lazydiff.vars import Scalar, Vector
from lazydiff import ops
import math

# creating variable x=3 and y=0
x = Scalar(3)
y = Scalar(0)

# constructing and evaluating f(x) = x^2 and f(y) = sin(y)
fx = x**2
fy = ops.sin(y)
# compound function f(z) = 5*z where z = f(x)
fz = 5*fx

# accessing the evaluated values
# output should print out 9, 0 and 45
print("Evaluating f(x) = x^2 where x = 3 : {}".format(fx.val))
print("Evaluating f(y) = siny where y = 0 : {}".format(fy.val))
print("Evaluating f(z) = 5*z where z = f(x) : {}".format(fz.val))

Evaluating f(x) = x^2 where x = 3 : 9
Evaluating f(y) = siny where y = 0 : 0.0
Evaluating f(z) = 5*z where z = f(x) : 45


### Evaluating Function Gradients

As each function is evaluated, the corresponding gradient is also stored in the new output variable. Therefore, each of the output variable contains the gradients from all the previously evaluated functions. This allows us to access the gradient with respect to any of the variables previously computed. For instance, for f(z) from the above example we can compute the gradient with respect to z or x depending on what we pass as input to the gradient method. This also makes it necessary to store the variables of interest somewhere in the program and then use the variable for which we want to evaluate the gradient of as input to the gradient method of the function. 

Below we show an example of how to access gradients of f(x), f(y), f(z) using `.grad(variable)` method

Note that `.grad()` can take multiple variables as input, returning the derivatives with respect to all these variables in the given order.

In [2]:
# accessing gradients
print("Gradient of f(x) w.r.t x is {}".format(fx.grad(x)))
print("Gradient of f(y) w.r.t y is {}".format(fy.grad(y)))
# gradient of f(z) w.r.t z is just 5 because d5z/dz = 5
# gradient of f(z) w.r.t x is d5x^2/dx = 10x = 30
# illustratation of taking two variables as input 
# and returning the gradient w.r.t. these two variables
print("Gradients of f(z) w.r.t z and x are {}".format(fz.grad(fx,x)))

Gradient of f(x) w.r.t x is [6.]
Gradient of f(y) w.r.t y is [1.]
Gradients of f(z) w.r.t z and x are [ 5. 30.]


### Working with Vectors

`Vector` objects can be instantiated from array of `Scalar` or from multiple `Scalar` arguments. There is no direct support for entering numeric values directly to `Vector` since it is necessary to create and store these `Scalar` for which we used as input for `Vector` to compute the gradient part as mentioned above.

Below we show an example of how to initialize `Vector` in two different ways:

In [3]:
# creating an array of Scalars
scalar1To5 = [Scalar(i+1) for i in range(5)]
# creating 3 different Scalar variables
s1,s2,s3,s4,s5= Scalar(1), Scalar(2), Scalar(3), Scalar(4), Scalar(5)

# instantiating with array of Scalars
x = Vector(scalar1To5)
# instantiating with 3 different Scalar variables as input arguments
y = Vector(s1,s2,s3,s4,s5)

`Vector` are handled in a similar manner as `Scalar` with similar corresponding operations.

Below we show an example evaluating the value and the gradients/Jacobian of $f(z) = 2*z$ where $z = f(x,y)$ and  $f(x,y) = x+y$ where is x,y are the vectors defined above.

In [4]:
# constructing and evaluating f(x,y) and f(z)
fxy = x+y
fz = 2*fxy

# accessing the evaluated values
print("Evaluating f(x,y) = x + y : {}".format(fxy.val))
print("Evaluating f(z) = 2*z : {}".format(fz.val))

# Jacobian of f(x,y) w.r.t x is an identity matrix
print("Jacobian of f(x,y) w.r.t. x:\n{}".format(fxy.grad(x)))
# Jacobian of f(z) w.r.t x is 2*identity matrix
print("Jacobian of f(z) w.r.t. x:\n{}".format(fz.grad(x)))

Evaluating f(x,y) = x + y : (2, 4, 6, 8, 10)
Evaluating f(z) = 2*z : (4, 8, 12, 16, 20)
Jacobian of f(x,y) w.r.t. x:
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
Jacobian of f(z) w.r.t. x:
[[2. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0.]
 [0. 0. 2. 0. 0.]
 [0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 2.]]


`Vector` also support `ops` operations just like `Scalar` do!

Below we show an example evaluating the value and the gradients/Jacobian of $f(z) = sin(z)$ where $z=f(x)$ and $f(x) = 2x$ where $x=[0,\frac{\pi}{2}]$ 

In [5]:
x = [Scalar(0), Scalar(math.pi/2)]
fx = 2*Vector(x)
fz = ops.sin(fx)
# 2*x should evaluate to (0,pi)
print("Evaluating f(x) = 2*x : {}".format(fx.val))
# sin(2x) should evaluate to (0,0)
print("Evaluating f(z) = sin(z) : {}".format(fz.val))

# Jacobian of f(x) w.r.t. x is 2*identity matrix
print("Jacobian of f(x) w.r.t. x:\n{}".format(fx.grad(x)))
# Jacobian of f(z) w.r.t. x has 2 and -2 on the diagonal
print("Jacobian of f(z) w.r.t. x:\n{}".format(fz.grad(x)))
# Jacobian of f(z) w.r.t. z has 1 and -1 on the diagonal
print("Jacobian of f(z) w.r.t. z:\n{}".format(fx.grad(fx)))

Evaluating f(x) = 2*x : (0, 3.141592653589793)
Evaluating f(z) = sin(z) : (0.0, 1.2246467991473532e-16)
Jacobian of f(x) w.r.t. x:
[[2. 0.]
 [0. 2.]]
Jacobian of f(z) w.r.t. x:
[[ 2.  0.]
 [ 0. -2.]]
Jacobian of f(z) w.r.t. z:
[[1. 0.]
 [0. 1.]]


### Easy-To-Use Newton's Method with lazzydiff

We show how we can use Newton's Method to solve for root of a question using lazzydiff

In this case, we explore this using $f(x) = x^2 - 2$, which has a root at $x=\sqrt2$

In [6]:
def Newton_Method(f, x0, maxiter=1000):
    """ returns the root for the given function
        using Newtons Method
        
        this function supports only Scalar
        you can modify this function to support Vector
        
        ===INPUT===
        f: function written in ops or built_in arithmetic functions
        x0: initial point
        maxiter: maximum number of iterations
    """
    x = x0
    for _ in range(maxiter):
        if abs(f(x)).val < 1e-8:
            return x
        fx = f(x)
        newx = x - fx/fx.grad(x)[0]
        if (abs(newx-x).val < 1e-8):
            return newx
        x = newx

In [7]:
# the function x^2-2 for which we want to find the root of
fx = lambda x: x**2-2
ans = Newton_Method(fx, Scalar(1))
print("The root found by Newton's Method using autodiff is: {}".format(ans.val))
print("The difference between this root and the real root 2**0.5 is : {}".format(ans.val-2**0.5))

The root found by Newton's Method using autodiff is: 1.4142135623746899
The difference between this root and the real root 2**0.5 is : 1.5947243525715749e-12


# Background
 AD exploits the fact that every function, no matter how complicated, is a sequence of elementary arithmetic operations (addition, subtraction, multiplication, division, etc.) and elementary functions (such as $\sin$, $\cos$, $\exp$, $\log$, etc.). By applying the chain rule repeatedly to these operations, derivatives of arbitrary order can be computed automatically, accurately to working precision, and using at most a small constant factor more arithmetic operations than the original program.
 * **Chain Rule**  \\
   - AD relies heavily on the chain rule from calculus, which says that $$\frac{d}{dx} f(g(x)) = \frac{df}{dg} \frac{dg}{dx} $$ For a more complex composition $$y = f(g(h(x))) = f(g(h(x_1))) = f(g(x_2)) = f(x_3) = x_4$$ the chain rule gives   \\
    $$\frac{dy}{dx} = \frac{dy}{dx_3} \frac{dx_3}{dx_2} \frac{dx_2}{dx_1}\frac{dx_1}{dx}$$  \\
    
 * **Graph structure of calculations**:  \\
 We can think of taking derivatives in AD as building a graph representing evaluation of a function, and using the chain rule to propagate derivative values from input variables to the final function, where each node represents an elementary operation or elementary function.
 
 Below is a sample such graph structure resulting from evaluating the function $$f(x_1,x_2) = \sin(x_1)  + x_1 x_2$$
 
 ![alt text](ForwardAccumulationAutomaticDifferentiation.png)
 [image source](https://upload.wikimedia.org/wikipedia/commons/a/a4/ForwardAccumulationAutomaticDifferentiation.png)
 
 
 We see that the input values $x_1,x_2$ and their derivatives $\dot{w_1}, \dot{w_2}$ propagate upward and are used in computations for the derivatives $\dot{w_3}, \dot{w_4}, \dot{w_5}$ at each node on the way up to the final function $f(x_1,x_2) = w_5$. 
 
<!--  ![alt text](https://upload.wikimedia.org/wikipedia/commons/a/a0/ReverseaccumulationAD.png =400x200)   \\ -->
 

  
 


# Software Organization

## Directory Structure

Our directory structure looks like the following:
* lazydiff/
     * `__init__.py`
     * `variables.py`
     * `ops.py`
     * tests/
       * `__init__.py`
       * `test_vars.py`
       * `test_ops.py`
       * `test_vector.py`
       * `test_vector_ops.py`
     * extensions/
       * `__init__.py`
       * `<relevant_classes>.py`
* docs/
 * milestone1.ipynb
 * milestone2.ipynb
* README.md
* LICENSE
* setup.py
* requirements.txt
* .travis.yml
* setup.cfg


All the modules related to autodiff are placed inside the `lazydiff` folder. Files outside this folder are miscellaneous files for setups, Github or test coverages.

The key modules included are `variables.py`, which contains our base classes that represent scalar/vector variables and built-in functions for which we may want to compute, and `ops.py`, which contains methods that allow us to evaluate elementary functions such as $\sin$, $\cos$, etc.

Additional extensions, such as reverse mode, backpropagation etc., will be included in the `extensions` folder. 

The test suite live in the folder "tests" in the directory structure above, and we use Travic CI for continuous integration and Coveralls for test coverage.

The packaged library is distributed through Github which can be installed using the pip command and the tutorial is available above in the `How to Install` section.

# Implementation

## Core Data Structures

The core of this library relies on the idea of variables, which store a float for the value it represents; a dictionary of previosly computed gradient values given variable reference as key; a list of previously computed variable references.

Through this data structure, we can easily find the gradient with respect to any past variable used without recomputing it each time. Please see the demo for an example of this.

Vector support is added by creating a vector class that stores a list of corresponding scalars. All the operations are performed on each of these scalars, which should be already implemented.

## Core Classes and Important Attributes

* In module `vars` :
    * We have a class `Scalar` which contains:
        * Constructor which takes in a value `val`
            * Initializes empty gradient dictionary `grad_val`, which will contain derivatives of this variable with respect to other variables.
            * Initializes list representing gradient calculation parents (tuples of parent variables and weight of the gradients of each parent). For instance, if we are evaluating the derivative of $\sin x$ at $x = 3$, $$\frac{d}{dx} \sin x \Big|_{x = 3}= \dot{x}|_{x=3} \cos 3$$ so the list would include the tuple $(x, \cos 3)$.
        * Method ```grad``` which takes in variables, and returns a ndarray containing the derivative with respect to those variables. 
        * Methods `__add__`, `__mul__` etc for overloading basic arithmetic operations
        
 * We have a class `Vector` which contains:
      * Constructor which takes in an array of `Scalar` or multiple `Scalar` arguments
       * Initializes a tuple of references to these `Scalar` instances to carry out corresponding operations on them when needed.
       * Initializes a tuple of values that these `Scalar` instances currently hold
      * Method ```grad``` which takes in variables of interest, and returns a ndarray representing the Jacobian with respect to those variables.
      * Methods `__add__`, `__mul__` etc for overloading basic arithmetic operations to support vector operations
      
## Elementary functions

* In module `ops`:
    * We have a  method for each elementary function (such as `sin`, `cos`, `exp`, etc.). Each method:
        * Take in a `Scalar` or `Vector` object - decorators are used to implement support for `Vector`
        * Create a new `Var` object with value updated to reflect execution of elementary function on value of input `Var` object, updated gradient calculation parent list - this compounds functions together
        
    * We have implemented all the basic arithmetic functions from `math` and `numpy` libraries, by directly calling these functions for evaluating the values. Thus, our `ops` library supports basic elementary functions ranging from `abs` to `asin` etc.
    
## External Dependencies

The implementations rely on `math` and `numpy` for evaluating elementary functions, such as cos, sin, tan. `numpy` is also used to support ndarray.

It relies on `numbers` and `collections` to check whether numeric values or containers (for `Vector`) have been entered as input.

## Not Implemented Yet

We still need to add support for seeding derivatives,  allowing to manually set the derivatives of a variable, i.e. a different value from the default value 1. 

We are also still considering how to pretty print Scalar or Vector objects in an informative way through `__repr__`.

# Future

## Reverse Mode

We are considering to implement reverse mode as one of the extensions. This can be done by storing children instead of parents with the corresponding changes, such as calling operation should append the derivative to the children. The main challenge here is to try reusing the operations `ops` or other methods we already have through perhaps decorators methods, trying to make minimal changes to implement reverse mode, rather than starting it from scratch. 

## Back Propagation

Once reverse mode is implemented, we believe that back propagation can be implemented easily by utilizing the properties of reverse mode. In this case, the main challenge still lies in implementing a reliable and working reverse mode first.

## Option for higher-order derivatives (Hessians and beyond)

We are thinking of supporting higher-order derivatives, such as Hessians. This can be done by further extending the Vector class with additional methods named as `.Hessian(variables)`. The main challenges of this extension is to figure out how to utilize `.grad(variables)` method to effectively get Hessians or beyond in a recursive manner to prevent reusing duplicate codes. Another challenge is how to do thorough testings of different operations on Hessian and beyond, which can be quite labor intensive.